In [2]:
####### Run a simple analysis of terrain around a selected location

# Step 1: Sign up for a Google Earth Engine Account here:
# https://earthengine.google.com/signup/

# you will use this to authenticate your account with GEE to run the script and
# utilize GEE layers

# Step 2: Setup a local conda environment on your machine to run Python and GEE

# Step 3: Install python, geemap, earthengine-api and pandas in your conda environment

# Step 4: 

In [3]:
# setup modules
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import pandas as pd
import sys,os,os.path,tempfile

# spatial modules
import ee
#import geemap # currently not using as colorbars dont load properly in google collab
import geemap.colormaps as cm
import geemap.foliumap as geemap # use folium instead so coloramps work on goolge collab
# https://geemap.org/notebooks/geemap_and_folium/

import warnings
warnings.filterwarnings('ignore') #don't display warnings

In [4]:
## setup output. defined dir or system temp dir
data_dir=os.path.expanduser(os.path.join(tempfile.gettempdir(),'uas_output'))

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print(" ")
print("****** Temporary Output Folder ******")
print("****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******")
print(data_dir)

 
****** Temporary Output Folder ******
****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******
/var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/uas_output


In [ ]:
##### initialize ee API  - you need to authenticate with GEE and initialize

## Trigger the authentication flow. You only need to do this once while running notebook
ee.Authenticate()

## After inserting the API key initialize GEE
ee.Initialize()

In [6]:
##### Link to Example Cloud Optimized GeoTiffs (COGs) located in a Google Cloud Storage "bucket"
# URL of file in the GCS bucket
URL = 'gs://bnl_uas_data/NGEEArctic_UAS_Kougarok_20180725_Flight6_RGB_cog.tif'
# using geemap load the COG file into memory
kg_rgb_image = geemap.load_GeoTIFF(URL)
# create a visualization layer for map display
RGB_imageVisParam = {"opacity":1,"bands":["B0","B1","B2"],
                    "min":-4.063213446967957,"max":85.22120105681381,
                     "gamma":1};

# URL of file in the GCS bucket
URL = 'gs://bnl_uas_data/NGEEArctic_UAS_Kougarok_20180725_Flight6_CHM_cog.tif'
kg_chm_image = geemap.load_GeoTIFF(URL)
kg_chm_image = kg_chm_image.select('B0').multiply(0.01)
chm_mask = kg_chm_image.gt(0);
chm_uas_image_masked = kg_chm_image.updateMask(chm_mask);
#print(kg_chm_image.getInfo())
print('CHM Info', kg_chm_image.getInfo()['bands'])
dem_palette = cm.get_palette('Spectral_r', n_class=30)
demViz = {'min': 0.0, 'max': 2.0, 'palette': dem_palette, 'opacity': 1}

# URL of file in the GCS bucket
URL = 'gs://bnl_uas_data/NGEEArctic_UAS_Kougarok_20180725_Flight6_TIR_cog.tif'
kg_tir_image = geemap.load_GeoTIFF(URL)
kg_tir_image = kg_tir_image.select('B0').multiply(0.1)
tir_mask = kg_tir_image.gt(0);
tir_uas_image_masked = kg_tir_image.updateMask(tir_mask);
tir_palette = cm.get_palette('Spectral_r', n_class=30)
tirViz = {'min': 15, 'max': 30, 'palette': tir_palette, 'opacity': 1}

CHM Info [{'id': 'B0', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [22850, 14148], 'crs': 'EPSG:32603', 'crs_transform': [0.01, 0, 508169.23, 0, -0.010000000000031596, 7226835.705]}]


In [7]:
##### Create a simple map displaying an RGB UAS image (Yang et al., 2021)

# center map over the Kougarok study site
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=17);
uas_map.addLayer(kg_rgb_image, RGB_imageVisParam, 'UAS_Kougarok_20180725_Flight6_RGB')
uas_map # creates a geemap.folium layer